In [116]:
# Jugador que se puede mover en cualquier lugar 
from dataclasses import dataclass, field
from typing import Dict, Union, List
import random as rd
from abc import ABC, abstractmethod

@dataclass
class Tiles(ABC):
    Name: str
    Position: List[int]
    def __post_init__(self):
        self.Position = self.Position

    @abstractmethod
    def Move(self):
        pass

    def CheckMovement(self, FuturePosition: int):
        if FuturePosition < 0 or FuturePosition > Grid.Size - 1:
            print('Invalid Movement')
            return False
        return True
    

@dataclass
class Player(Tiles):
    Health: int = 100
    Inventory= {
        "Medicine": 0,
        "Weapons": 0
    }

    def __post_init__(self):
        self.Position = self.Position

    def GetInventory(self):
        return (f'{self.Name} has {self.Inventory["Medicine"]} Medicines and {self.Inventory["Weapons"]} Weapons')
    
    def Move(self, Direction: str):
        if Direction.lower() == 'up' and self.CheckMovement(self.Position[0] - 1):
            self.Position[0] -= 1
        elif Direction.lower() == 'down' and self.CheckMovement(self.Position[0] + 1):
            self.Position[0] += 1
        elif Direction.lower() == 'left' and self.CheckMovement(self.Position[1] -1):
            self.Position[1] -= 1
        elif Direction.lower() == 'right' and self.CheckMovement(self.Position[1] + 1):
            self.Position[1] += 1
        else:
            print('Invalid Direction')
            return
        print(f'{self.Name} moved to {self.Position}')

    

@dataclass
class InventoryObjects(Tiles):
    
    def  UpdateInventory(self, Player: Player):
        pass

@dataclass
class Medicine(InventoryObjects):
    HealRegen: int
    def Move(self):
        pass
    def UpdateInventory(self, Player: Player):
        if Player.Health < 100:
            Player.Health += 10
            print(f'{Player.Name} has been healed by 10 points')
        else:
            Player.Inventory['Medicine'] += 1
            print(f'{Player.Name} has picked up a Medicine')  

@dataclass
class Weapons(InventoryObjects):
    Damage: int
    def Move(self):
        pass
    def UpdateInventory(self, Player: Player):
        Player.Inventory['Weapons'] += 1
        print(f'{Player.Name} has picked up a Weapon')

    
@dataclass
class Wall(Tiles):
    
    def Move(self):
        pass

    






@dataclass
class Grid:
    Size: int = 15
    Positions: List = field(default_factory=list)
    Walls: List = field(default_factory=list)



    def __post_init__(self):
        self.grid = [[' ' for _ in range(self.Size)] for _ in range(self.Size)] 

    def AddPlayer(self, Player: Player):
        self.Positions.append(Player)
    
    def Environment(self):
        for _ in range(10):    
            WallPosition = [rd.randint(0, self.Size - 1), rd.randint(0, self.Size - 1)]
            self.Walls.append(Wall('W', WallPosition))
            self.Positions.append(Medicine('M', [rd.randint(0, self.Size - 1), rd.randint(0, self.Size - 1)], 7))
            self.Positions.append(Weapons('A', [rd.randint(0, self.Size - 1), rd.randint(0, self.Size - 1)], 10))


    def CreateEnvironment(self):
        self.Environment()
        for Wall in self.Walls:
            self.grid[Wall.Position[0]][Wall.Position[1]] = Wall.Name
    


    def UpdatePosition(self):
            self.grid = [[' ' for _ in range(self.Size)] for _ in range(self.Size)] 
            for Position in self.Positions:
                    self.grid[Position.Position[0]][Position.Position[1]] = Position.Name
            for Wall in self.Walls:
                self.grid[Wall.Position[0]][Wall.Position[1]] = Wall.Name               

    def GetElement(self, Position: List[int]):
        return self.grid[Position[0]][Position[1]]

    def UpdateGrid(self):
        self.UpdatePosition()
        self.ShowGrid()

    def ShowGrid(self):
        for row in self.grid:
            print(row)

    def ayuda(self):
        return self.Walls
    
@dataclass
class Game:
    Grid = Grid()
    Elements = ["P", "W", "M"]



    def MapLimits(self, PlayerPos):
        if PlayerPos[0] < 0 or PlayerPos[0] > 14 or PlayerPos[1] < 0 or PlayerPos[1] > 14:
            print('Movement blocked by map limits')
            return False
        return True
        

    def GetElements(self, Player: Player, Direction: str):
        PosibbleNewPos = {
            "up" : [-1,0],
            "down" : [1,0],
            "left" : [0,-1],
            "right" : [0,1],
        }
        pos = PosibbleNewPos[Direction]
        new_position = [Player.Position[0] + pos[0], Player.Position[1] + pos[1]]

        for Element in self.Grid.Positions:
            if Element.Position == new_position:
                Element.UpdateInventory(Player)
                Player.Move(Direction)
                self.Grid.Positions.remove(Element)
                break
        


    def MovePlayer(self, Player: Player, Direction: str):
        PosibbleNewPos = {
            "up" : [-1,0],
            "down" : [1,0],
            "left" : [0,-1],
            "right" : [0,1],
        }
        pos = PosibbleNewPos[Direction]
        new_position = [Player.Position[0] + pos[0], Player.Position[1] + pos[1]]

        if  self.MapLimits(new_position):
        
            if  self.Grid.GetElement(new_position) == 'W':
                print('Movement blocked by a wall')
            elif self.Grid.GetElement(new_position) == 'M':
                self.GetElements(Player, Direction)
            elif self.Grid.GetElement(new_position) == 'A':
                self.GetElements(Player, Direction)

            else: 
                Player.Move(Direction)
                print(self.Grid.GetElement(new_position))
        else:
            print('Invalid Movement')



        



In [117]:
Juego = Game()
Player1 = Player('P', [2,2])
Juego.Grid.AddPlayer(Player1)
Juego.Grid.CreateEnvironment()


In [160]:
Juego.MovePlayer(Player1, 'down')
print(Player1.GetInventory())
Juego.Grid.UpdateGrid()

Movement blocked by map limits
Invalid Movement
P has 2 Medicines and 2 Weapons
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', 'M', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', 'A', ' ', ' ', 'W', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['A', ' ', ' ', ' ', 'A', ' ', 'A', ' ', ' ', ' ', ' ', ' ', ' ', 'M', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', 'W', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', 'W', ' ', ' ', ' ', ' ', 'W', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'A', 'M', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'W', ' ', ' ', ' ', 'M', ' ', ' ', ' ', ' ', 'W', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'M', ' ']
['A', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', 'M', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'W', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' 

[Wall(Name='W', Position=[2, 8]),
 Wall(Name='W', Position=[7, 5]),
 Wall(Name='W', Position=[6, 7]),
 Wall(Name='W', Position=[9, 6]),
 Wall(Name='W', Position=[1, 6]),
 Wall(Name='W', Position=[8, 6]),
 Wall(Name='W', Position=[5, 0]),
 Wall(Name='W', Position=[6, 8]),
 Wall(Name='W', Position=[1, 1]),
 Wall(Name='W', Position=[3, 4])]